In [35]:
import subprocess
import pandas as pd
from tqdm import tqdm

In [29]:
# returns a list of users for an AD group
def users(group):
    result = subprocess.run(f"net group /domain {group}", shell=True, capture_output=True, text=True).stdout
    result = result.split("\n")
    output = []
    for r in result:
        if "." in r and "cmpd1.metoffice.gov.uk" not in r and "successfully" not in r:
            row = r.split(" ")
            for name in row:
                if name != '':
                    output.append(name)

    return output

In [50]:
# set groupt
groups = [
    "PG_Operators",
    "PG_Meteogram_Users",
    "PG_Meteogram_Managing",
    "PG_Developers_TstEnv",
    "PG_Developers_QA",
    "PG_Developers_prd",
    "PG_Developers_dev",
    "PG_Developers",
    "PG_Administrators",
    "PG_Scheduler_admin_QA",
    "PG_Scheduler_admin_prd",
    "PG_Scheduler_admin_dev",
    "PG_Operators_TstEnv",
    "PG_Operators_QA",
    "PG_Operators_prd",
    "PG_Operators_dev]"
    ]
groups.sort()

In [51]:
# get data (by name)
data = {}
for i in tqdm(range(len(groups))):
    group = groups[i]
    names = names = users(groups[i])
    for name in names:
        if name in data:
            data[name].append(group)
        else:
            data[name] = [group]

100%|██████████| 16/16 [00:14<00:00,  1.11it/s]


In [62]:
# generate dataframe
df = pd.DataFrame(columns=["user"]+groups)
for user in data:
    row = [user]
    for group in groups:
        if group in data[user]:
            row.append("True")
        else:
            row.append(None)
    df.loc[len(df)] = row

df.head()

,user,PG_Administrators,PG_Developers,PG_Developers_QA,PG_Developers_TstEnv,PG_Developers_dev,PG_Developers_prd,PG_Meteogram_Managing,PG_Meteogram_Users,PG_Operators,PG_Operators_QA,PG_Operators_TstEnv,PG_Operators_dev],PG_Operators_prd,PG_Scheduler_admin_QA,PG_Scheduler_admin_dev,PG_Scheduler_admin_prd
0,alice.ranford,True,True,None,True,None,None,None,None,None,None,None,None,None,None,None,None
1,andrew.robertson,True,True,None,None,None,None,None,None,None,None,True,None,None,None,None,None
2,andy.butler,True,True,None,True,True,None,None,None,None,None,None,None,None,None,None,None
3,ashley.ryall,True,True,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,bridget.george,True,True,None,True,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
# save to excel
df.to_excel("PG_AD_Groups.xlsx", index=False)